In [ ]:
import import_ipynb
import player as P
import pygame
import sys
import time

## PLAY AGAINST

In [ ]:
def playoff(bot_white,bot_black):
    WIDTH, HEIGHT = 600, 600
    GRID_SIZE = 8
    CELL_SIZE = WIDTH // GRID_SIZE
    BLACK = (0, 0, 0)
    WHITE = (255, 255, 255)
    GREEN = (0, 128, 0)
    # Initialize the game board
    board = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
    board[3][4] = board[4][3] = 2  # White
    board[3][3] = board[4][4] = 1  # Black
    # Current player (1 for White, 2 for Black)
    # Initialize Pygame
    # # Create the game window

    # pygame.init()
    # pygame.font.init() 
    # my_font = pygame.font.SysFont('Comic Sans MS', 10)

    # screen = pygame.display.set_mode((WIDTH, HEIGHT))
    # pygame.display.set_caption("8x8 Othello")

    # def draw_board():
    #     screen.fill(GREEN)
    #     for x in range(GRID_SIZE):
    #         for y in range(GRID_SIZE):
    #             pygame.draw.rect(screen, BLACK, (x*CELL_SIZE, y*CELL_SIZE, CELL_SIZE, CELL_SIZE), 1)
    #             if board[y][x] == 1:
    #                 pygame.draw.circle(screen, WHITE, (x*CELL_SIZE + CELL_SIZE//2, y*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2 - 5)
    #             elif board[y][x] == 2:
    #                 pygame.draw.circle(screen, BLACK, (x*CELL_SIZE + CELL_SIZE//2, y*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2 - 5)

    flag = 1
    current_player = 2
    while flag:
        
        flag = 0
        for x in range(GRID_SIZE):
            for y in range(GRID_SIZE):
                if P.is_valid_move(x,y,board,current_player): flag=1

        if current_player == 1 and flag:
            x,y = bot_white.get_move(board,current_player)    
            if P.is_valid_move(x, y,board,current_player):
                P.make_move(x,y,board,current_player)
            current_player = 2
        
        elif current_player == 2 and flag:
            
            grid_x,grid_y = bot_black.get_move(board,current_player)
            if P.is_valid_move(grid_x, grid_y,board,current_player):
                P.make_move(grid_x, grid_y,board,current_player)
                current_player = 1

        # Quit = False
        # for event in pygame.event.get():
        #     if event.type == pygame.QUIT:
        #         Quit = True    
        # if(Quit): break

        if not flag:
            wins = P.decide_winner(board,current_player)
     
    #     draw_board()
    #     pygame.display.flip()
    # pygame.quit()
    return wins



In [ ]:
mcts = P.MCTSPlayer(explorationFactor=1,rollouts=300,selectionDepth=2)
alpha = P.AlphaBetaPlayer(depth = 3)
rando = P.RandomPlayer()

In [ ]:
players = [mcts,alpha,rando]
elo = [1200,1200,1200]

def match(idxWhite,idxBlack):
    E_w = 1/(1 + 10**((elo[idxBlack] - elo[idxWhite])/400))
    E_b = 1/(1 + 10**((elo[idxWhite] - elo[idxBlack])/400))

    blackWin = playoff(bot_white = players[idxWhite],bot_black = players[idxBlack])

    if(blackWin):
        elo[idxBlack] = elo[idxBlack] + 32*(1 - E_w)
        elo[idxWhite] = elo[idxWhite] + 32*(0 - E_b)
    else:
        elo[idxBlack] = elo[idxBlack] + 32*(0 - E_w)
        elo[idxWhite] = elo[idxWhite] + 32*(1 - E_b)

In [ ]:
for x in range(100):
    for i in range(3):
        match(i%3,(i+1)%3)
        print(".",end="")
    for i in range(3):
        match((i+1)%3,(i)%3)
        print(".",end="")
    print(elo)